In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from ase.io import read,write
from ase.visualize import view
import numpy as np 

from equistore import Labels, TensorBlock, TensorMap
from itertools import product
from equistore_utils.clebsh_gordan import ClebschGordanReal

import matplotlib.pyplot as plt

# from equistore_utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
# from rascal.representations import SphericalExpansion
from equistore_utils.acdc_mini import acdc_standardize_keys, _remove_suffix
from rascaline import SphericalExpansion
from rascaline import SphericalExpansionByPair as PairExpansion
from equistore import operations
import copy
import ase 
import tqdm 
import hickle

In [3]:
from equistore_utils.mp_utils import *

In [4]:
from ase.io import read

In [5]:
# frames_all = read('bispectrum_counter/carbon/C-VII-pp-wrapped.xyz', ':10000')
# leng = np.asarray([len(f) for f in frames_all])
# frames_14 = []
# for idx in np.where(leng<14)[0]:
# #     print(idx)
#     frames_14.append(frames_all[idx])
# frames = frames_14
# write('bispectrum_counter/carbon/C_lessthan14.xyz', frames)
# print(np.max([np.max(f.get_all_distances()) for f in frames_14]))
frames = read('bisp/C_lessthan14.xyz', ':10')

In [51]:
def acdc_standardize_keys(descriptor,drop_pair_id=False):
    """ Standardize the naming scheme of density expansion coefficient blocks (nu=1) """
    key_names = np.array(descriptor.keys.names)
    if not "spherical_harmonics_l" in key_names:
        raise ValueError("Descriptor missing spherical harmonics channel key `spherical_harmonics_l`")
    if "species_atom_1" in key_names:
        key_names[np.where(key_names=="species_atom_1")[0]] = "species_center"
    if "species_atom_2" in key_names:
        key_names[np.where(key_names=="species_atom_2")[0]] = "species_neighbor"
    key_names = tuple(key_names)
    blocks = []
    keys = []
    for key, block in descriptor:
        key = tuple(key)
        if not "inversion_sigma" in key_names:
            key = (1,) + key 
        if not "order_nu" in key_names:
            key = (1,) + key 
        keys.append(key)
        property_names = _remove_suffix(block.properties.names, "_1")
        sample_names = [ "center" if b == "first_atom" else ("neighbor" if b == "second_atom" else b) for b in block.samples.names ]
#         drops pair_id which we don't need in this application
        if "pair_id" in sample_names and drop_pair_id:
            new_samples = block.samples.view(dtype = np.int32).copy().reshape(-1, len(sample_names))
            print("sample", new_samples)
            icent = np.where(np.asarray(sample_names)=="center")[0]
            ineigh = np.where(np.asarray(sample_names)=="neighbor")[0]
            ipid = np.where(np.asarray(sample_names)=="pair_id")[0]
            new_samples[:,ineigh] += new_samples[:,ipid]*new_samples[:,icent]#.max()
            print(new_samples[:,icent].max())
            print("icent", icent, "ineigh", ineigh, "sample_neihj", new_samples[:,ineigh], print(new_samples[:, icent]))
            new_samples = Labels( [n for n in sample_names if n!="pair_id"],
                        new_samples[:,[i for i in range(len(block.samples.names)) if block.samples.names[i] != "pair_id"]])    
            
#             print(np.asarray(block.samples.view(dtype = np.int32)).reshape(-1,len(sample_names))[:,[i for i in range(len(block.samples.names)) if block.samples.names[i] != "pair_id"]])
#             new_samples = Labels( [n for n in sample_names if n!="pair_id"],
            
#                                  np.asarray(block.samples.view(dtype = np.int32)).reshape(-1,len(sample_names))[:,[i for i in range(len(block.samples.names)) if block.samples.names[i] != "pair_id"]])
        else:
            new_samples = Labels(sample_names, np.asarray(block.samples.view(dtype = np.int32)).reshape(-1,len(sample_names)) )
        blocks.append(
            TensorBlock( values=block.values, samples=new_samples,
                        components=block.components, 
                         properties=Labels(property_names, np.asarray(block.properties.view(dtype = np.int32)).reshape(-1,len(property_names)) )
            ) )
        
    if not "inversion_sigma" in key_names:
        key_names = ("inversion_sigma",) + key_names
    if not "order_nu" in key_names:
        key_names = ( "order_nu",) + key_names
    
    return TensorMap(keys = Labels(names=key_names, values=np.asarray(keys, dtype=np.int32)),
                     blocks = blocks
                    )

In [57]:
calculator = PairExpansion(**hypers)
gij = calculator.compute(frames[:])
# gij = operations.slice(gij, samples=Labels(['first_atom'],np.array([[0]], np.int32)) )
gij =  acdc_standardize_keys(gij,True)
gik = calculator.compute(frames[:])
gik = acdc_standardize_keys(gik,False)
# for b in gij.blocks():
# #     b.values[:]/=norm_rho1
# # gij = simplify_gij(gij)

sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
11
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample_neihj [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None
sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
11
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample_neihj [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None
sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
11
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample_neihj [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None
sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
11
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample_neihj [[ -11]
 [ -10]
 [  -9]

In [70]:
aa=np.unique(gij.block(0).samples["neighbor"].view(np.int32))

In [76]:
gij.block(0).samples["neighbor"].view(np.int32)

Labels([ -11,  -10,   -9, ..., 2832, 2843, 2854], dtype=int32)

In [75]:
aa

Labels([ -11,  -10,   -9, ..., 5608, 5619, 5630], dtype=int32)

In [77]:
aa= gik.block(0).samples.view(dtype = np.int32).copy().reshape(-1, 4)

In [91]:
gik.block(0)

TensorBlock
    samples (6564): ['structure', 'pair_id', 'center', 'neighbor']
    components (1): ['spherical_harmonics_m']
    properties (4): ['n_1']
    gradients: no

In [93]:
aa

Labels([[  0,  -1,   0,   0],
        [  0,  -1,   1,   1],
        [  0,  -1,   2,   2],
        ...,
        [  9, 257,   5,   5],
        [  9, 258,   5,   5],
        [  9, 259,   5,   5]], dtype=int32)

In [67]:
yy = aa[np.where(aa[:, 2]==0)[0]]
yy[np.where(yy[:, 3]==0)[0]]

Labels([[ 0, -1,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  2,  0,  0],
        [ 0,  3,  0,  0],
        [ 1, -1,  0,  0],
        [ 1,  0,  0,  0],
        [ 1,  1,  0,  0],
        [ 1,  2,  0,  0],
        [ 1,  3,  0,  0],
        [ 1,  4,  0,  0],
        [ 1,  5,  0,  0],
        [ 1,  6,  0,  0],
        [ 1,  7,  0,  0],
        [ 1,  8,  0,  0],
        [ 1,  9,  0,  0],
        [ 2, -1,  0,  0],
        [ 2,  0,  0,  0],
        [ 2,  1,  0,  0],
        [ 2,  2,  0,  0],
        [ 2,  3,  0,  0],
        [ 2,  4,  0,  0],
        [ 2,  5,  0,  0],
        [ 3, -1,  0,  0],
        [ 3,  0,  0,  0],
        [ 3,  1,  0,  0],
        [ 3,  2,  0,  0],
        [ 3,  3,  0,  0],
        [ 3,  4,  0,  0],
        [ 3,  5,  0,  0],
        [ 3,  6,  0,  0],
        [ 3,  7,  0,  0],
        [ 3,  8,  0,  0],
        [ 3,  9,  0,  0],
        [ 4, -1,  0,  0],
        [ 4,  0,  0,  0],
        [ 4,  1,  0,  0],
        [ 4,  2,  0,  0],
        [ 4,

## Set up basic features

In [28]:
from rascaline import SphericalExpansion
from rascaline import SphericalExpansionByPair as PairExpansion

In [29]:
hypers = {
    "cutoff": 5,
    "max_radial": 4,
    "max_angular": 3,
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.1}},
    "center_atom_weight": 0.0,    
}
cg = ClebschGordanReal(5)


In [30]:
calculator = SphericalExpansion(**hypers)
rhoi = calculator.compute(frames[:])
rhoi = rhoi.keys_to_properties(['species_neighbor'])
rho1i = acdc_standardize_keys(rhoi)
# rho1i = operations.slice(rho1i, samples=Labels(['center'],np.array([[0]], np.int32)) )
# norm_rho1 = np.sqrt(np.sum([(b.values**2).sum(axis=(1,2)) for b in rho1i.blocks()],axis=0).mean())
# for b in rho1i.blocks():
#     b.values[:]/=norm_rho1
    

In [10]:
rho2i = cg_combine(rho1i, rho1i, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=3)

In [11]:
rho3i =  cg_combine(rho2i, rho1i, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=0)

In [12]:
len(frames)

10

In [50]:
calculator = PairExpansion(**hypers)
gij = calculator.compute(frames[:])
# gij = operations.slice(gij, samples=Labels(['first_atom'],np.array([[0]], np.int32)) )
gij =  acdc_standardize_keys(gij,True)
# for b in gij.blocks():
# #     b.values[:]/=norm_rho1
# # gij = simplify_gij(gij)

sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None
sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None
sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None
sample [[  0  -1   0   0]
 [  0  -1   1   1]
 [  0  -1   2   2]
 ...
 [  9 257   5   5]
 [  9 258   5   5]
 [  9 259   5   5]]
[[0]
 [1]
 [2]
 ...
 [5]
 [5]
 [5]]
icent [2] ineigh [3] sample [[ -11]
 [ -10]
 [  -9]
 ...
 [2832]
 [2843]
 [2854]] None


In [48]:
gij.block(0).samples

Labels([(0,  -1, 0, 0), (0,  -1, 1, 1), (0,  -1, 2, 2), ...,
        (9, 257, 5, 5), (9, 258, 5, 5), (9, 259, 5, 5)],
       dtype=[('structure', '<i4'), ('pair_id', '<i4'), ('center', '<i4'), ('neighbor', '<i4')])

In [14]:
test = operations.sum_over_samples(gij, sample_names=["neighbor", "pair_id"]) 
np.linalg.norm(rho1i.block(0).values/test.block(0).values -1)

0.0

## Compute three-feats

In [15]:
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=3)


KeyboardInterrupt



In [26]:
rhoii1i2_nu0

TensorMap with 7 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_neighbor_a' 'species_neighbor_b']
           2             1                    0                   6                 6                   6
           2             1                    1                   6                 6                   6
           2             1                    2                   6                 6                   6
           2             1                    3                   6                 6                   6
           2            -1                    1                   6                 6                   6
           2            -1                    2                   6                 6                   6
           2            -1                    3                   6                 6                   6

In [16]:
rhoii1i2_nu0.block(0).samples

Labels([(0,  -1, 0, 0,  -1, 0), (0,  -1, 0, 0,   0, 0),
        (0,  -1, 0, 0,   1, 0), ..., (9, 259, 5, 5, 257, 5),
        (9, 259, 5, 5, 258, 5), (9, 259, 5, 5, 259, 5)],
       dtype=[('structure', '<i4'), ('pair_id1', '<i4'), ('center', '<i4'), ('neighbor_1', '<i4'), ('pair_id2', '<i4'), ('neighbor_2', '<i4')])

In [28]:
a = rhoii1i2_nu0
b = rhoii1i2_nu0

In [33]:
a.block(0)

TensorBlock
    samples (509430): ['structure', 'pair_id1', 'center', 'neighbor_1', 'pair_id2', 'neighbor_2']
    components (1): ['spherical_harmonics_m']
    properties (64): ['n_1_a', 'k_2', 'n_1_b', 'l_2']
    gradients: no

In [32]:
c.block(0)

TensorBlock
    samples (1018860): ['structure', 'pair_id1', 'center', 'neighbor_1', 'pair_id2', 'neighbor_2', 'tensor']
    components (1): ['spherical_harmonics_m']
    properties (64): ['n_1_a', 'k_2', 'n_1_b', 'l_2']
    gradients: no

In [30]:
c=operations.join([a,b], axis="samples")

In [17]:
gij.block(0).samples.names

('structure', 'pair_id', 'center', 'neighbor')

In [27]:
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, other_keys_match = ['species_center'], lcut=0)

KeyboardInterrupt: 

In [ ]:
rhoii1i2_nu1.block(0)

## Set up energy model 

In [ ]:
def StructureMap(samples_structure, device="cpu"):
    unique_structures, unique_structures_idx = np.unique(
        samples_structure, return_index=True
    )   
    new_samples = samples_structure[np.sort(unique_structures_idx)]
    # we need a list keeping track of where each atomic contribution goes
    # (e.g. if structure ids are [3,3,3,1,1,1,6,6,6] that will be stored as
    # the unique structures [3, 1, 6], structure_map will be
    # [0,0,0,1,1,1,2,2,2]
    replace_rule = dict(zip(unique_structures, range(len(unique_structures))))
    structure_map = torch.tensor(
        [replace_rule[i] for i in samples_structure],
        dtype=torch.long,
        device=device,
    )   
    return structure_map, new_samples, replace_rule